In [1]:
import torch
import torchvision
from torch import nn 
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image 
#%matplotlib inline
import numpy as np
import dataset_utils
import importlib
importlib.reload(dataset_utils)


<module 'dataset_utils' from '/home/fogliodicarta/Desktop/InsectClassification/dataset_utils.py'>

In [2]:
import modelC as m
def extract_features(self,x):
    x = self.main(x)
    x = self.flatten(x)
    return x
m.Discriminator.extract_features = extract_features

discriminator = m.Discriminator()
discriminator.cuda()
generator = m.Generator(noise_size=100)
generator.cuda()

Generator(
  (embed): Embedding(1050, 1000)
  (main): Sequential(
    (0): Tpose(
      (main): Sequential(
        (0): ConvTranspose2d(1100, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
    )
    (1): Tpose(
      (main): Sequential(
        (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
    )
    (2): Tpose(
      (main): Sequential(
        (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
      )
    )
   

In [3]:
g_dict = torch.load('checkpoints/generatorC')
generator.load_state_dict(g_dict['model_state_dict'])
d_dict = torch.load('checkpoints/discriminatorC')
discriminator.load_state_dict(d_dict['model_state_dict'])

<All keys matched successfully>

In [4]:
df = pd.read_csv('final_dataset.csv',index_col=0)
tform = transforms.Compose([transforms.Resize((64,64)),transforms.PILToTensor(),transforms.ConvertImageDtype(torch.float),transforms.Normalize(0.5,0.5)])
image_dataset = torchvision.datasets.ImageFolder("image_dataset/",transform=tform)
species2genus = dataset_utils.species_label_to_genus_label(df,image_dataset)
#label_genus_da_controllare = species2genus[specie_fissata]


In [5]:
batch_size = 5000
import random
import dataset_utils
img2dna = dataset_utils.get_imgs_bold_id(image_dataset,df)

nucleotides = df[['nucleotide','species_name','genus_name','processid','image_urls']]
colonna_dna = df.loc[:,"nucleotide"]
nucleotides.loc[:,'nucleotide'] = colonna_dna.apply(dataset_utils.one_hot_encoding)
random.seed(42)

X_train_1, X_test, y_train_1, y_test = dataset_utils.data_split(nucleotides,0.2,random_state=42)
print(y_test)
train_data = X_train_1
train_data['species_name'] = y_train_1

X_train, X_validation, y_train, y_validation = dataset_utils.data_split(train_data,0.2,drop_labels=False,random_state=42)
train_indices, val_indices, test_indices = dataset_utils.image_splits_from_df(X_train,X_validation,X_test,image_dataset)
from torch.utils.data import Dataset, DataLoader
 
class WholeDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.targets = data.targets#torch.tensor(targets)
        #self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index][0]
        y = self.targets[index]
        
        return x, y
    
    def __len__(self):
        return len(self.data)
        
whole_dataset = WholeDataset(image_dataset)
n_classes = np.unique(whole_dataset.targets).shape[0]
print(len(test_indices+val_indices+train_indices))
len(whole_dataset)

train_imgs = torch.utils.data.Subset(whole_dataset, train_indices)
val_imgs = torch.utils.data.Subset(whole_dataset, val_indices)
test_imgs = torch.utils.data.Subset(whole_dataset, test_indices)
len(train_imgs)+len(val_imgs)+len(test_imgs)
train_loader = torch.utils.data.DataLoader(train_imgs, batch_size=batch_size,shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_imgs, batch_size=2000,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_imgs, batch_size=4,shuffle=True, num_workers=2)
del test_loader
dataloaders = {"train":train_loader,"val":val_loader}
dataset_sizes = {'train': len(train_imgs.indices), 'val':len(val_imgs.indices)}


243       Acleris chalybeana
170        Apamea amputatrix
371       Malocampa matralis
328           Hemiceras soso
249       Eristalis stipator
               ...          
223          Telegonus talus
85        Sphecodes miniatus
122    Micropsectra radialis
459       Chionodes dentella
338       Halictus maculatus
Name: species_name, Length: 5296, dtype: object
32424


In [6]:
discriminator.eval()

with torch.no_grad():
    for batch,targets in dataloaders['train']:
        #print(type(batch))
        features = discriminator.extract_features(batch.cuda()) 
        features_targets = targets
        break


In [7]:
y= features_targets.cpu().numpy()
X = features.cpu().numpy()

In [50]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier 
#svm = LinearSVC(random_state=42)

# Make it an Multilabel classifier
#multilabel_classifier = MultiOutputClassifier(svm, n_jobs=-1)

# Fit the data to the Multilabel classifier
#svm = svm.fit(X, y)

# Get predictions for test data
#y_test_pred = svm.predict(X)
clf = RandomForestClassifier(min_samples_leaf=2,n_jobs=-1)
clf = clf.fit(X, y)
#n_estimators = 1
#clf = (BaggingClassifier(RandomForestClassifier(min_samples_leaf=50), max_samples=1.0 / n_estimators, n_estimators=n_estimators,n_jobs=10))
#clf = clf.fit(X,y)

In [51]:
y_train_pred = clf.predict(X)
np.count_nonzero(y_train_pred==y)/5000


1.0

In [52]:
torch.cuda.empty_cache()


In [53]:

discriminator.eval()

with torch.no_grad():
    for batch,targets in dataloaders['val']:
        #print(type(batch))
        val_features = discriminator.extract_features(batch.cuda()) 
        val_features_targets = targets
        break

In [54]:

y_val = val_features_targets.cpu().numpy()
X_val = val_features.cpu().numpy()

In [55]:
y_val_pred= clf.predict(X_val)
np.count_nonzero(y_val_pred==y_val)/2000

0.297

In [36]:
set(y_validation.unique())

{'Abax parallelus',
 'Acleris chalybeana',
 'Acleris macdunnoughi',
 'Acordulecera dorsalis',
 'Acrolophus forbesi',
 'Acrolophus kearfotti',
 'Acronicta alni',
 'Acronicta increta',
 'Acronicta modica',
 'Acronicta rumicis',
 'Adelpha iphiclus',
 'Adelpha leucophthalma',
 'Adelpha lorzae',
 'Adelpha melanthe',
 'Adelpha salmonides',
 'Adrastus pallens',
 'Aedes aegypti',
 'Aedes albopictus',
 'Aedes euplocamus',
 'Agabus bipustulatus',
 'Agabus sturmii',
 'Agabus uliginosus',
 'Agonum emarginatum',
 'Agonum lugens',
 'Agonum marginatum',
 'Agonum micans',
 'Agonum muelleri',
 'Agonum viduum',
 'Agriophara discobola',
 'Agriophara dyscapna',
 'Agriophara plagiosema',
 'Agriotes lineatus',
 'Agriotes pallidulus',
 'Agrotis ipsilon',
 'Agrotis puta',
 'Agrotis segetum',
 'Agrotis venerabilis',
 'Agrypnus murinus',
 'Aleochara bipustulata',
 'Aloconota gregaria',
 'Alosterna tabacicolor',
 'Amara aenea',
 'Amara apricaria',
 'Amara aulica',
 'Amara bifrons',
 'Amara consularis',
 'Amara c

In [40]:
len(val_indices)

6955

In [ ]:

len(val_indices)